In [37]:
import os, sys, re
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

sys.path.append('..')
import trilabtracker as tt

from importlib import reload
for m in tt.__all__:
    eval(f'reload(tt.{m})')
reload(tt)

os.getcwd()

'/media/yaouen/LarvalSchooling/larval_schooling/trilab-tracker/data-management'

## Prepare trial data.

Prepare a dictionary of trials to analyze with basic info for each (path to trial file, population, age, number of individuals, etc).

In [35]:
tank_diameter_vs_age = { 7:9.6, 14:10.4, 21:12.8, 28:17.7, 42:33.8,
                         56:33.8, 70:33.8, 84:3.8 }

# Extract trial metadata from the trial's filename.
def parse_trial_file(trial_file, etho=False):
    if etho:
        trial_dir  = None
        trial_name = os.path.basename(trial_file)
        trial_name = trial_name.split('-')[1]
    else:
        trial_dir  = os.path.dirname(trial_file)
        trial_name = os.path.basename(trial_dir)
    pop,day,age,group,n_ind = trial_name.split('_')[:5]
    pop        = {'sf':'SF', 'pa':'Pa', 'rc':'RC'}[pop.lower()]
    age        = int(age[:-3])
    age = 42 if age==43 else (70 if age==71 else age)
    n_ind      = int(re.findall('\d+',n_ind)[0])
    R_cm       = tank_diameter_vs_age[age]/2
    trial      = { k:v for k,v in locals().items() if k in ['trial_file', 'trial_dir', 
                                 'trial_name', 'pop', 'age', 'group', 'n_ind', 'R_cm'] }
    return trial

# Select a set of trials to analyze.
trial_files = sorted(glob('../output/*/trial.pik'))
# print(trial_files)

# Count trials of each type.
trials = [ parse_trial_file(f) for f in trial_files]
trials = pd.DataFrame(trials,index=trial_files)
grouped_trials = trials.groupby(['pop','age','n_ind'])
count  = pd.DataFrame(grouped_trials['trial_dir'].count().rename('count'))
count = count.unstack(1)
count.columns = count.columns.droplevel()
count[pd.isna(count)] = 0
count = count.astype(int)
display(count)

def matching_trials(pop=None, age=None, n_ind=None, df=trials):
    I = pd.Series(data=True, index=df.index)
    if not pop is None:
        I = I & (df['pop']==pop)
    if not age is None:
        I = I & (df['age']==age)
    if not n_ind is None:
        I = I & (df['n_ind']==n_ind)
    return df[I].index.tolist()

,age,7
pop,n_ind,
SF,5,1


## Visualize unfixed and fixed trajectories around a fix.

In [3]:
# trial_file = matching_trials(age=28)[0]
# print(trial_file)

# trial = tt.load_trial(trial_file, load_fixes=False)
# data1 = trial['data'].copy()
# trial = tt.load_trial(trial_file, load_fixes=True)
# data2 = trial['data'].copy()

# plt.figure(figsize=(6,6))
# I,J = slice(6050,6150), [0,2,4]
# for j in J:
#     c = f'C{j}'
#     plt.plot(*data1[I,j,:2].T, lw=8, alpha=0.2, color=c, zorder=-5)
#     plt.plot(*data2[I,j,:2].T, lw=1, color=c)
# # plt.axis('equal')
# plt.show()

## Convert to excel.

In [61]:
from importlib import reload
for m in tt.__all__:
    eval(f'reload(tt.{m})')
reload(tt)

for trial_file in tqdm(trials.index):
    trial = parse_trial_file(trial_file)
    trial = tt.preprocess_trial(trial)
    globals().update(trial)
    
    fx = f'../excel/{trial_name}.xlsx'
    print(fx)
    if os.path.exists(fx):
        continue
    
    writer = pd.ExcelWriter(fx)
    for i in range(n_ind):
        df = pd.DataFrame(np.concatenate([time[:,None],data[:,i,:3]], axis=1), 
                          columns=['Time', 'X (cm)', 'Y (cm)', 'Direction (rad)'])
        df.to_excel(writer, sheet_name=f'fish {i+1}', index=None)
    for sheet in writer.sheets.values():
        for col in sheet.columns:
            for cell in col:
                if isinstance(cell.value,float):
                    cell.number_format = '0.000'
            sheet.column_dimensions[col[0].column_letter].width = 15
    writer.save()
    break

100%|██████████| 1/1 [00:00<00:00, 33.73it/s]

../output/sf_thur_7dpf_groupA_n5b_20210513_2021-05-13-142401/sf_thur_7dpf_groupA_n5b_20210513_2021-05-13-142401_new.xlsx
